In [1]:
from database import *
import psycopg2
from HST_variations import *
from psycopg2.extras import register_json
import pandas as pd
from io import StringIO
import os
import pandas as pd
import json
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
dbHost = '13.36.221.162'
dbName = 'thminsight'
dbUser = 'thminsight'
dbPort = '5432'
dbPassword = 'psS8hqr6oaoJ7fgk'

In [3]:
db=Database(dbHost, dbName, dbUser,dbPassword,dbPort )
queryManager = Query(dbHost, dbName, dbUser,dbPort, dbPassword)

Database Ready
Database Ready


In [4]:
id_temperature_var = 1245
id_water_level_var = 1246
id_tv_var=1238
id_new_var=1238
id_new_var2=1238
id_res_var = 0
res_metric_id = 47
res_name = 'res'

In [5]:
##step 1 : choose target column
target_column= 'target_variable'
df_def = queryManager.get_df_by_variable_id(id_tv_var, target_column)
target_variable_name=queryManager.get_variable_name_by_id(id_tv_var)

## step2 : choose model variables (Variables explicatives)

df_temp = queryManager.get_df_by_variable_id(id_temperature_var,'temperature')
df_H = queryManager.get_df_by_variable_id(id_water_level_var, 'water_level')


##somee added variables to model ##potentielly
var_name=queryManager.get_variable_name_by_id(id_new_var)
var_name2=queryManager.get_variable_name_by_id(id_new_var2)


##add variables explicatives to model 
df_V=queryManager.get_df_by_variable_id(id_new_var, var_name)
df_V1=queryManager.get_df_by_variable_id(id_new_var2, var_name2)

In [6]:
model=['H','S']

In [7]:
dataframes=[df_def,df_H]

In [8]:
df=final_prep(dataframes)
df.head()

,Time,target_variable,water_level
1,2022-02-18 05:08:44,-1.0,5.380181
2,2022-02-23 01:48:13,-1.0,5.379942
3,2022-02-22 08:28:17,-1.0,5.379704
4,2022-02-22 22:28:14,-1.0,5.377601
5,2022-02-23 14:08:10,-1.0,5.375498


In [9]:
len(df)

9887

In [10]:
optimizers_list=['SGD','Adam', 'AdamW','Adamax','ASGD','Adagrad']
methods_list=['std_regression',
              'std_regression_cross',
              'use_NN_optimizer',
              'use_NN_optimizer_cross',
             'use_bayesian_approch']

In [11]:
for method,optimizer in zip(methods_list,optimizers_list):
    print(method,optimizer)
    res=deploy(model,df,target_column,method,optimizer)
res.head()

std_regression SGD
std_regression_cross Adam
use_NN_optimizer AdamW
use_NN_optimizer_cross Adamax
use_bayesian_approch ASGD


,corrected observation,Time,target_variable,water_level
1,7.225331e-13,2022-02-18 05:08:44,-1.0,5.380181
2,7.231993e-13,2022-02-23 01:48:13,-1.0,5.379942
3,7.029932e-13,2022-02-22 08:28:17,-1.0,5.379704
4,7.054357e-13,2022-02-22 22:28:14,-1.0,5.377601
5,7.056578e-13,2022-02-23 14:08:10,-1.0,5.375498


In [12]:
res.shape

(9887, 4)

In [13]:
id_res_var=180
df_res =prepare_corrected_obs(res,id_res_var)

In [14]:
df_res.head()

,timestamp,value,variable_id
1,2022-02-18 05:08:44,7.225331e-13,180
2,2022-02-23 01:48:13,7.231993e-13,180
3,2022-02-22 08:28:17,7.029932e-13,180
4,2022-02-22 22:28:14,7.054357e-13,180
5,2022-02-23 14:08:10,7.056578e-13,180


In [16]:
queryManager.deleteData(id_res_var, '1967-01-01 00:00:00.00+02:00')
queryManager.db.insertDataframe(df_res, 'raw_data_save') 

Insert dataframe done with success!
